# Building embeddings from our corpus

Now we have our corpus, we can build our embeddings.

There are different types of embeddings. At our corpus we will work with word embeddings. However we could think of sentence embeddings, paragraph embeddings, document embeddings, even character or subword embeddings.

Word embeddings are vector representations of words. They are useful for many NLP tasks, such as sentiment analysis, text classification, and machine translation. They are also useful for visualising the relationships between words. For example, we can use them to find words with similar meanings, or to find words that are often used together.

## Why do we need embeddings?

Computers and most machine learnign algorithms in particular work better with numerical data. However, words are not numbers. So we need to find a way to represent words as numbers.

Large Language Models such as ChatGPT also use word embeddings to represent words. However, they are trained on huge amounts of data, and so are not always suitable for smaller projects. In this notebook, we will build our own word embeddings from our corpus.

## Different types of embedddings

There are different types of word embeddings as well, such as:

- Word2Vec
- GloVe
- FastText
- BERT

 We will  also use the gensim library to do this. Gensim is a library for topic modelling, document indexing and similarity retrieval with large corpora. It uses the word2vec algorithm to create vector representations of words, which can then be used to find words with similar meanings.